In [1]:
from tabnet import TabNet, TabNetClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from statistics import mean
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from statistics import mean
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split


In [2]:
df = pd.read_csv('../data/JMpreprocessed_data2.csv')

In [3]:
#splitting up data into training/testing
y = df['Winner']
X = df.drop(columns = 'Winner')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=43)

In [4]:
scores_TabNet = []

In [5]:
clf = TabNetClassifier()  #TabNetRegressor()
kf = KFold(n_splits=3)
counter = 0
for train_index, test_index in kf.split(X_train, y_train):
    print("counter: " + str(counter))
    counter+=1
    xtrain = X_train.iloc[min(train_index):max(train_index)]
    xtest = X_train.iloc[min(test_index):max(test_index)]
    ytrain = y_train.iloc[min(train_index):max(train_index)]
    ytest = y_train.iloc[min(test_index):max(test_index)]
    clf.fit(
      X_train.to_numpy(), y_train.to_numpy(),
      eval_set=[(X_test.to_numpy(), y_test.to_numpy())],
      eval_metric=['auc'],
      max_epochs=150 , patience=15,
      batch_size=128, virtual_batch_size=64,
      num_workers=0,
      weights=1,
      drop_last=False
    )
    
    y_preds = clf.predict(xtest.to_numpy())
    
    scores_TabNet.append(accuracy_score(ytest, y_preds))

Device used : cpu
counter: 0
epoch 0  | loss: 0.75559 | val_0_auc: 0.53095 |  0:00:02s
epoch 1  | loss: 0.69224 | val_0_auc: 0.48684 |  0:00:04s
epoch 2  | loss: 0.68712 | val_0_auc: 0.51963 |  0:00:07s
epoch 3  | loss: 0.68304 | val_0_auc: 0.58384 |  0:00:09s
epoch 4  | loss: 0.68282 | val_0_auc: 0.57433 |  0:00:11s
epoch 5  | loss: 0.67767 | val_0_auc: 0.53649 |  0:00:14s
epoch 6  | loss: 0.68127 | val_0_auc: 0.55737 |  0:00:16s
epoch 7  | loss: 0.67715 | val_0_auc: 0.56772 |  0:00:18s
epoch 8  | loss: 0.6784  | val_0_auc: 0.58808 |  0:00:20s
epoch 9  | loss: 0.67473 | val_0_auc: 0.61025 |  0:00:22s
epoch 10 | loss: 0.66986 | val_0_auc: 0.62233 |  0:00:25s
epoch 11 | loss: 0.66186 | val_0_auc: 0.59923 |  0:00:27s
epoch 12 | loss: 0.65728 | val_0_auc: 0.60303 |  0:00:29s
epoch 13 | loss: 0.65991 | val_0_auc: 0.59256 |  0:00:32s
epoch 14 | loss: 0.6507  | val_0_auc: 0.62809 |  0:00:34s
epoch 15 | loss: 0.63968 | val_0_auc: 0.63257 |  0:00:37s
epoch 16 | loss: 0.63255 | val_0_auc: 0.615

In [6]:
mean(scores_TabNet)

0.6394817073170732

In [7]:
#Testing
y_preds = clf.predict(X_test.to_numpy())

In [8]:
accuracy_score(y_test, y_preds)

0.5496575342465754

In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_preds)

array([[176, 212],
       [ 51, 145]])